In [1]:
import re,os
import numpy as np
import random
from math import log
mypath="authors"

In [2]:
def dataSet(dirName):
    authors=[]
    path=os.getcwd()+'/'+dirName+'/'
    mylist=[]
    for filename in os.listdir(path):
        authors.append(filename)
        for file in os.listdir(path+filename):
            files=open(path+filename+'/'+file,'r',encoding="utf-8")
            text=files.read().split()
            mylist.append(text) 
    return mylist,authors

In [3]:
x=[]
import time
t1 =time.time()
dataset,authors=dataSet(mypath)
dataset=[[w.lower() for w in line] for line in dataset] #lowercase
for doc in dataset:
    sentence=" ".join(doc)
    x.append(re.sub(r"[^\w\d'\s]+",'',sentence).split(" "))

dataset=x
t2=time.time()
print(t2-t1)


0.7388811111450195


In [4]:
docData=[]
docAuthors=[]
defaultDocWord=530
for i in range(len(authors)):
    lenBook=len(dataset[i])
    lenDoc=int(lenBook/defaultDocWord)
    if(lenBook%defaultDocWord!=0):
        lenDoc=lenDoc+1
    for k in range(lenDoc):
        docData.append(dataset[i][k*defaultDocWord:(k+1)*defaultDocWord])
        docAuthors.append(authors[i]) 


In [5]:

def divideDataSet(docData,docAuthors):
   
    suffl = list(zip(docData, docAuthors))
    random.shuffle(suffl)
    docData, docAuthors = zip(*suffl)

    lenTrainSet=int(len(docData)*0.8)
    
    trainSetData=docData[0:lenTrainSet]
    trainSetAuthors=docAuthors[0:lenTrainSet]
    
    testSetData=docData[lenTrainSet:len(docData)]
    testSetAuthors=docAuthors[lenTrainSet:len(docAuthors)]
    return trainSetData,trainSetAuthors,testSetData,testSetAuthors


In [6]:
trainSetData,trainSetAuthors,testSetData,testSetAuthors = divideDataSet(docData,docAuthors) #get train and test value

In [7]:
from collections import Counter
def counter(docData):
    wordcount = Counter(docData)
    return wordcount

In [8]:
def joinWord():
    authSet=[]
    for auth in authors:
        index=[]
        wordSet=[]
        index = [index for index, value in enumerate(trainSetAuthors) if value == auth]
        for i in index:
            wordSet.extend(trainSetData[i])
        authSet.append(wordSet)
    return authSet

In [9]:
#Distinct Word joining
authSet=joinWord() # Kelimelerin sınıflandırılmış listesi
auth_word=[counter(authSet[i]) for i in range(len(authors))] #Distinct kelimeler

In [10]:
def match_list(docData):
    len_auth=len(authors)
    matching_list=[]
    for word in docData:
        tmp_list=[]
        for i in range(len_auth):
                tmp_list.append(auth_word[i][word])
        matching_list.append(tmp_list)
    return matching_list
    

In [11]:
tmp_list=match_list(testSetData[0])
#Smoothing
def smoothing(tmp_list):
    for i in range(len(authors)):
        for k in range(len(tmp_list)):
            tmp_list[k][i]=(tmp_list[k][i]+1)/(len(authSet[i])+len(auth_word[i]))
    return tmp_list

In [12]:

def prob(matrix):
    
    prob_list=np.zeros(len(authors))
    lenAuthors=len(authors)
    lenMatrix=len(matrix)
    for k in range(lenMatrix):
        for i in range(lenAuthors):
            sum_row=sum(matrix[k])
            prob_list[i]=prob_list[i]+log(matrix[k][i]/sum_row)
    return prob_list
    
    

In [13]:
prob_list=[]
#max probability calculate
for i in range(len(testSetData)):
    matrix=smoothing(match_list(testSetData[i]))
    
    prob_list.append(prob(matrix)) 
print(prob_list)



[array([-1157.7672272 , -1165.93778837, -1320.52760811, -1356.45443855,
       -1320.78446372, -1197.56862918,  -822.08558618]), array([-1202.76057815, -1087.62379058, -1368.48846992, -1339.43911287,
       -1189.1827653 ,  -921.05245892, -1125.76771839]), array([ -891.78421368, -1114.56669009, -1263.32211112, -1312.17091022,
       -1304.06284984, -1299.6008795 , -1174.24515832]), array([-1404.26770072, -1416.59875431,  -710.47971597, -1582.70773403,
       -1582.37853793, -1546.14741536, -1472.70652425]), array([-1165.46087765,  -838.53067915, -1319.34977039, -1321.26633623,
       -1169.36959464, -1224.38268124, -1208.61315247]), array([-1133.16575169, -1076.50302479, -1328.65861148, -1092.64679469,
       -1187.62263642, -1146.32271387, -1063.54143062]), array([-1297.37876055, -1110.455067  , -1423.85650846, -1402.04929122,
        -782.54122049, -1295.23543821, -1200.80670713]), array([-1222.30071665,  -805.65958958, -1395.52701817, -1373.0410673 ,
       -1231.18821823, -1228.051

In [14]:

def external_test(file):
        files=open(file,'r',encoding="utf-8")
        test=files.read().split()
        matrix=prob(smoothing(match_list(test)))
        max_ind=np.argmax(matrix)
        print(authors[max_ind],"-",file)
    

In [15]:
external_test("james joyce.txt")

Ahmet hamdi - james joyce.txt


In [16]:
def conf_matrix(cos_list):
    conf_mat=np.zeros((len(authors),len(authors)))
    for i in range(len(cos_list)):
        max_in=np.argmax(cos_list[i])
        #print(authors[max_in]," : ", testSetAuthors[i])
        conf_mat[authors.index(testSetAuthors[i])][max_in]=conf_mat[authors.index(testSetAuthors[i])][max_in]+1

    return conf_mat

In [17]:
matrix=conf_matrix(prob_list)
print(matrix)

[[ 49.   0.   0.   0.   0.   0.   0.]
 [  0.  29.   0.   0.   0.   0.   0.]
 [  0.   0.  46.   0.   0.   0.   0.]
 [  5.  39.   0.  46.   0.   0.   7.]
 [  0.   0.   0.   0.  48.   0.   0.]
 [  0.   1.   0.   0.   0.  64.   0.]
 [  0.   0.   0.   0.   0.   0.  23.]]


In [18]:
def  pre_rec(matrix,i):
    colm_pred=sum(matrix[:,i])
    row_pred=sum(matrix[i])
    prec=matrix[i][i]/colm_pred
    recall=matrix[i][i]/row_pred
    return prec,recall

In [19]:

def f1_score():
    prec_average=0
    recall_average=0
    lenAuthors=len(authors)
    for i in range(lenAuthors):
        prec,recall=pre_rec(matrix,i)
        prec_average=prec_average+prec/lenAuthors
        recall_average=recall_average+recall/lenAuthors
    f1_score=2*(prec_average*recall_average)/(prec_average+recall_average)
    return f1_score

In [20]:
f1_score()

0.89590168643435752

In [21]:
def accuracy(matrix):
    all_pred=sum(sum(matrix))
    t_pred=sum(np.diagonal(matrix))
    return (t_pred/all_pred)

In [22]:
accuracy(matrix)


0.85434173669467783